Import pandas and use Selenium to scrape data for the S&P 500, limit the scraping to about 1 year by putting a 365 cap on it

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (confusion_matrix, roc_curve, roc_auc_score, ConfusionMatrixDisplay)

In [2]:
# Set up  browser
options = Options()

driver = webdriver.Chrome(options=options)
driver.get("https://finance.yahoo.com/quote/%5EGSPC/history/?period1=1277683200&period2=1743107763")

time.sleep(5)

# Get the rows from the desired table
rows = driver.find_elements(By.XPATH, "//table[contains(@class, 'yf-1jecxey')]/tbody/tr")

# open a data dictionary
data = []
i = 0
while len(data) < 1300 and i < len(rows):
    row_text = rows[i].text
    cells = row_text.split()

    if len(cells) >= 9:
        date = " ".join(cells[0:3])
        values = cells[3:9]
        full_row = [date] + values
        data.append(full_row)
    
    i += 1

driver.quit()

# Create a DataFrame and add all the desired columns
GSPC_df = pd.DataFrame(data, columns=['Date', 'Open', 'High', 'Low', 'Close*', 'Adj Close**', 'Volume'])
print(GSPC_df)

              Date      Open      High       Low    Close* Adj Close**  \
0     Mar 27, 2025  5,695.64  5,732.28  5,670.94  5,693.31    5,693.31   
1     Mar 26, 2025  5,771.66  5,783.62  5,694.41  5,712.20    5,712.20   
2     Mar 25, 2025  5,775.96  5,786.95  5,760.42  5,776.65    5,776.65   
3     Mar 24, 2025  5,718.08  5,775.14  5,718.08  5,767.57    5,767.57   
4     Mar 21, 2025  5,630.73  5,670.84  5,603.10  5,667.56    5,667.56   
...            ...       ...       ...       ...       ...         ...   
1295  Jan 31, 2020  3,282.33  3,282.33  3,214.68  3,225.52    3,225.52   
1296  Jan 30, 2020  3,256.45  3,285.91  3,242.80  3,283.66    3,283.66   
1297  Jan 29, 2020  3,289.46  3,293.47  3,271.89  3,273.40    3,273.40   
1298  Jan 28, 2020  3,255.35  3,285.78  3,253.22  3,276.24    3,276.24   
1299  Jan 27, 2020  3,247.16  3,258.85  3,234.50  3,243.63    3,243.63   

             Volume  
0     4,403,460,000  
1     4,183,140,000  
2     4,467,340,000  
3     4,517,990,000  
4

In [3]:
GSPC_df

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Mar 27, 2025","5,695.64","5,732.28","5,670.94","5,693.31","5,693.31","4,403,460,000"
1,"Mar 26, 2025","5,771.66","5,783.62","5,694.41","5,712.20","5,712.20","4,183,140,000"
2,"Mar 25, 2025","5,775.96","5,786.95","5,760.42","5,776.65","5,776.65","4,467,340,000"
3,"Mar 24, 2025","5,718.08","5,775.14","5,718.08","5,767.57","5,767.57","4,517,990,000"
4,"Mar 21, 2025","5,630.73","5,670.84","5,603.10","5,667.56","5,667.56","9,367,460,000"
...,...,...,...,...,...,...,...
1295,"Jan 31, 2020","3,282.33","3,282.33","3,214.68","3,225.52","3,225.52","4,529,700,000"
1296,"Jan 30, 2020","3,256.45","3,285.91","3,242.80","3,283.66","3,283.66","3,790,350,000"
1297,"Jan 29, 2020","3,289.46","3,293.47","3,271.89","3,273.40","3,273.40","3,600,250,000"
1298,"Jan 28, 2020","3,255.35","3,285.78","3,253.22","3,276.24","3,276.24","3,531,570,000"


In [4]:
GSPC_df.columns = GSPC_df.columns.str.replace('*', '').str.replace('**', '').str.strip()

GSPC_df = GSPC_df.rename(columns={col: f'GSPC_{col}' for col in GSPC_df.columns if col != 'Date'})

In [5]:
GSPC_df

,Date,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Adj Close,GSPC_Volume
0,"Mar 27, 2025","5,695.64","5,732.28","5,670.94","5,693.31","5,693.31","4,403,460,000"
1,"Mar 26, 2025","5,771.66","5,783.62","5,694.41","5,712.20","5,712.20","4,183,140,000"
2,"Mar 25, 2025","5,775.96","5,786.95","5,760.42","5,776.65","5,776.65","4,467,340,000"
3,"Mar 24, 2025","5,718.08","5,775.14","5,718.08","5,767.57","5,767.57","4,517,990,000"
4,"Mar 21, 2025","5,630.73","5,670.84","5,603.10","5,667.56","5,667.56","9,367,460,000"
...,...,...,...,...,...,...,...
1295,"Jan 31, 2020","3,282.33","3,282.33","3,214.68","3,225.52","3,225.52","4,529,700,000"
1296,"Jan 30, 2020","3,256.45","3,285.91","3,242.80","3,283.66","3,283.66","3,790,350,000"
1297,"Jan 29, 2020","3,289.46","3,293.47","3,271.89","3,273.40","3,273.40","3,600,250,000"
1298,"Jan 28, 2020","3,255.35","3,285.78","3,253.22","3,276.24","3,276.24","3,531,570,000"


In [6]:
GSPC_df.dtypes

Date              object
GSPC_Open         object
GSPC_High         object
GSPC_Low          object
GSPC_Close        object
GSPC_Adj Close    object
GSPC_Volume       object
dtype: object

In [7]:
for col in GSPC_df.columns:
    if col != 'Date':
        GSPC_df[col] = pd.to_numeric(GSPC_df[col].str.replace(',', ''), errors='coerce')

In [8]:
GSPC_df['Date'] = pd.to_datetime(GSPC_df['Date'], errors='coerce')


In [9]:
GSPC_df.dtypes

Date              datetime64[ns]
GSPC_Open                float64
GSPC_High                float64
GSPC_Low                 float64
GSPC_Close               float64
GSPC_Adj Close           float64
GSPC_Volume                int64
dtype: object

In [10]:
GSPC_df['GSPC_Return'] = (GSPC_df['GSPC_Adj Close'] - GSPC_df['GSPC_Open']) / GSPC_df['GSPC_Open']
GSPC_df

,Date,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Adj Close,GSPC_Volume,GSPC_Return
0,2025-03-27,5695.64,5732.28,5670.94,5693.31,5693.31,4403460000,-0.000409
1,2025-03-26,5771.66,5783.62,5694.41,5712.20,5712.20,4183140000,-0.010302
2,2025-03-25,5775.96,5786.95,5760.42,5776.65,5776.65,4467340000,0.000119
3,2025-03-24,5718.08,5775.14,5718.08,5767.57,5767.57,4517990000,0.008655
4,2025-03-21,5630.73,5670.84,5603.10,5667.56,5667.56,9367460000,0.006541
...,...,...,...,...,...,...,...,...
1295,2020-01-31,3282.33,3282.33,3214.68,3225.52,3225.52,4529700000,-0.017308
1296,2020-01-30,3256.45,3285.91,3242.80,3283.66,3283.66,3790350000,0.008356
1297,2020-01-29,3289.46,3293.47,3271.89,3273.40,3273.40,3600250000,-0.004882
1298,2020-01-28,3255.35,3285.78,3253.22,3276.24,3276.24,3531570000,0.006417


In [11]:
GSPC_df['GSPC_Return'] = GSPC_df['GSPC_Return'] * 100
GSPC_df

,Date,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Adj Close,GSPC_Volume,GSPC_Return
0,2025-03-27,5695.64,5732.28,5670.94,5693.31,5693.31,4403460000,-0.040908
1,2025-03-26,5771.66,5783.62,5694.41,5712.20,5712.20,4183140000,-1.030206
2,2025-03-25,5775.96,5786.95,5760.42,5776.65,5776.65,4467340000,0.011946
3,2025-03-24,5718.08,5775.14,5718.08,5767.57,5767.57,4517990000,0.865500
4,2025-03-21,5630.73,5670.84,5603.10,5667.56,5667.56,9367460000,0.654089
...,...,...,...,...,...,...,...,...
1295,2020-01-31,3282.33,3282.33,3214.68,3225.52,3225.52,4529700000,-1.730783
1296,2020-01-30,3256.45,3285.91,3242.80,3283.66,3283.66,3790350000,0.835572
1297,2020-01-29,3289.46,3293.47,3271.89,3273.40,3273.40,3600250000,-0.488226
1298,2020-01-28,3255.35,3285.78,3253.22,3276.24,3276.24,3531570000,0.641713


In [12]:
TSLA_df = pd.read_csv('TSLA.csv')
TSLA_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,6/29/2010,1.266667,1.666667,1.169333,1.592667,1.592667,281494500
1,6/30/2010,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2,7/1/2010,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
3,7/2/2010,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
4,7/6/2010,1.333333,1.333333,1.055333,1.074000,1.074000,103003500


In [13]:
TSLA_df['Date'] = pd.to_datetime(TSLA_df['Date'])
TSLA_df.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [14]:
TSLA_df = TSLA_df.rename(columns={col: f'TSLA_{col}' for col in TSLA_df.columns if col != 'Date'})
TSLA_df

,Date,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500
...,...,...,...,...,...,...,...
3675,2025-02-06,373.029999,375.399994,363.179993,374.320007,374.320007,77918200
3676,2025-02-07,370.190002,380.549988,360.339996,361.619995,361.619995,70298300
3677,2025-02-10,356.209991,362.700012,350.510010,350.730011,350.730011,77514900
3678,2025-02-11,345.799988,349.369995,325.100006,328.500000,328.500000,118543400


In [15]:
TSLA_df['TSLA_Return'] = (TSLA_df['TSLA_Adj Close'] - TSLA_df['TSLA_Open']) / TSLA_df['TSLA_Open']
TSLA_df['TSLA_Return'] = TSLA_df['TSLA_Return'] * 100
TSLA_df

,Date,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,TSLA_Return
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500,25.736833
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500,-7.599808
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000,-12.160018
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000,-16.521720
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500,-19.449981
...,...,...,...,...,...,...,...,...
3675,2025-02-06,373.029999,375.399994,363.179993,374.320007,374.320007,77918200,0.345819
3676,2025-02-07,370.190002,380.549988,360.339996,361.619995,361.619995,70298300,-2.315029
3677,2025-02-10,356.209991,362.700012,350.510010,350.730011,350.730011,77514900,-1.538413
3678,2025-02-11,345.799988,349.369995,325.100006,328.500000,328.500000,118543400,-5.002888


In [16]:
TSLA_df.dtypes

Date              datetime64[ns]
TSLA_Open                float64
TSLA_High                float64
TSLA_Low                 float64
TSLA_Close               float64
TSLA_Adj Close           float64
TSLA_Volume                int64
TSLA_Return              float64
dtype: object

In [17]:
merged_df = pd.merge(TSLA_df, GSPC_df, on='Date', how='inner')
merged_df

,Date,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,TSLA_Return,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Adj Close,GSPC_Volume,GSPC_Return
0,2020-01-27,36.132668,37.629333,35.952000,37.201332,37.201332,204121500,2.957613,3247.16,3258.85,3234.50,3243.63,3243.63,3831050000,-0.108710
1,2020-01-28,37.899334,38.453999,37.205334,37.793331,37.793331,176827500,-0.279696,3255.35,3285.78,3253.22,3276.24,3276.24,3531570000,0.641713
2,2020-01-29,38.379333,39.320000,37.828667,38.732666,38.732666,267022500,0.920632,3289.46,3293.47,3271.89,3273.40,3273.40,3600250000,-0.488226
3,2020-01-30,42.161331,43.391998,41.200001,42.720669,42.720669,435085500,1.326660,3256.45,3285.91,3242.80,3283.66,3283.66,3790350000,0.835572
4,2020-01-31,42.666668,43.533333,42.167999,43.371334,43.371334,235789500,1.651561,3282.33,3282.33,3214.68,3225.52,3225.52,4529700000,-1.730783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,2025-02-06,373.029999,375.399994,363.179993,374.320007,374.320007,77918200,0.345819,6072.22,6084.03,6046.83,6083.57,6083.57,4847120000,0.186917
1266,2025-02-07,370.190002,380.549988,360.339996,361.619995,361.619995,70298300,-2.315029,6083.13,6101.28,6019.96,6025.99,6025.99,4766900000,-0.939319
1267,2025-02-10,356.209991,362.700012,350.510010,350.730011,350.730011,77514900,-1.538413,6046.40,6073.38,6044.84,6066.44,6066.44,4458760000,0.331437
1268,2025-02-11,345.799988,349.369995,325.100006,328.500000,328.500000,118543400,-5.002888,6049.32,6076.28,6042.34,6068.50,6068.50,4324880000,0.317060


In [18]:
merged_df.isna().sum()

Date              0
TSLA_Open         0
TSLA_High         0
TSLA_Low          0
TSLA_Close        0
TSLA_Adj Close    0
TSLA_Volume       0
TSLA_Return       0
GSPC_Open         0
GSPC_High         0
GSPC_Low          0
GSPC_Close        0
GSPC_Adj Close    0
GSPC_Volume       0
GSPC_Return       0
dtype: int64

In [19]:
merged_df.to_csv('tesla&market_returns2.csv', index=False)

In [20]:
# set up browser options
speakrj_options = Options()

# setup driver
speakrj_driver = webdriver.Chrome(options=speakrj_options)
speakrj_driver.get("https://www.speakrj.com/audit/report/elonmusk/twitter/summary/2020-03-23/2025-04-22")

time.sleep(5)

# get only data rom the specific table
summary_rows = speakrj_driver.find_elements(By.XPATH, "//table[@class='table table-responsive-md']/tbody/tr")

# get data from each row
summary_data = []
for row in summary_rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    row_data = [col.text.strip() for col in cols]
    if row_data:
        summary_data.append(row_data)

speakrj_driver.quit()

# create df with raw data 
elon_raw_df = pd.DataFrame(summary_data)

# Preview the first few rows
print(elon_raw_df.head())

            0    1            2            3    4     5       6       7  \
0  2023-05-22  Mon  140,475,025     +608,106  306    +7  26,147    +206   
1  2023-05-17  Wed  139,866,919   +6,721,230  299  +113  25,941  +1,914   
2  2023-03-31  Fri  133,145,689  +12,219,109  186   +42  24,027  +2,828   
3  2022-12-11  Sun  120,926,580   +1,192,310  144   +12  21,199    +301   
4  2022-12-01  Thu  119,734,270     +313,947  132    +2  20,898     +50   

        8       9  
0  24,016    +432  
1  23,584  +3,356  
2  20,228  +4,391  
3  15,837    +345  
4  15,492     +95  


In [21]:
elon_raw_df

,0,1,2,3,4,5,6,7,8,9
0,2023-05-22,Mon,"140,475,025","+608,106",306,+7,"26,147",+206,"24,016",+432
1,2023-05-17,Wed,"139,866,919","+6,721,230",299,+113,"25,941","+1,914","23,584","+3,356"
2,2023-03-31,Fri,"133,145,689","+12,219,109",186,+42,"24,027","+2,828","20,228","+4,391"
3,2022-12-11,Sun,"120,926,580","+1,192,310",144,+12,"21,199",+301,"15,837",+345
4,2022-12-01,Thu,"119,734,270","+313,947",132,+2,"20,898",+50,"15,492",+95
...,...,...,...,...,...,...,...,...,...,...
342,2020-04-27,Mon,221,-,156,-,177,-,80,-
343,2020-04-26,Sun,221,-,156,-,177,-,80,-
344,2020-04-25,Sat,221,-,156,-,177,-,80,-
345,2020-04-24,Fri,221,-,156,-,177,-,80,-


In [22]:
elon_raw_df.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
7    object
8    object
9    object
dtype: object

In [23]:
elon_cleaned_df = elon_raw_df[[0, 3]].copy()
elon_cleaned_df.columns = ['Date', 'change_in_followers']

In [24]:
elon_cleaned_df['Date'] = pd.to_datetime(elon_cleaned_df['Date'], errors='coerce')
#replace unwanted characters
elon_cleaned_df['change_in_followers'] = elon_cleaned_df['change_in_followers'].str.replace('+', '')
elon_cleaned_df['change_in_followers'] = elon_cleaned_df['change_in_followers'].str.replace(',', '')
elon_cleaned_df['change_in_followers'] = pd.to_numeric(elon_cleaned_df['change_in_followers'], errors='coerce')

In [25]:
elon_cleaned_df = elon_cleaned_df.dropna()

elon_cleaned_df = elon_cleaned_df.reset_index(drop=True)

In [26]:
elon_cleaned_df

,Date,change_in_followers
0,2023-05-22,608106.0
1,2023-05-17,6721230.0
2,2023-03-31,12219109.0
3,2022-12-11,1192310.0
4,2022-12-01,313947.0
...,...,...
323,2020-05-13,174066.0
324,2020-05-10,33958013.0
325,2020-05-07,1.0
326,2020-05-01,-1.0


In [27]:
elon_cleaned_df.isna().sum()

Date                   0
change_in_followers    0
dtype: int64

In [28]:
elon_cleaned_df.to_csv('elon_followers.csv', index = False)

In [29]:
final_merged_df = pd.merge(merged_df, elon_cleaned_df, on='Date', how='inner')

In [30]:
final_merged_df

,Date,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,TSLA_Return,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Adj Close,GSPC_Volume,GSPC_Return,change_in_followers
0,2020-04-29,52.678001,53.546665,52.210667,53.367332,53.367332,243240000,1.308575,2918.46,2954.86,2912.16,2939.51,2939.51,6645640000,0.721271,-1.0
1,2020-05-01,50.333332,51.518002,45.535999,46.754665,46.754665,487977000,-7.109934,2869.09,2869.09,2821.61,2830.71,2830.71,4759810000,-1.337706,-1.0
2,2020-05-07,51.813999,53.093334,51.490002,52.002666,52.002666,172915500,0.364124,2878.26,2901.92,2876.48,2881.19,2881.19,5178790000,0.101798,1.0
3,2020-05-13,54.722000,55.066666,50.886665,52.730667,52.730667,285982500,-3.638999,2865.86,2874.14,2793.15,2820.00,2820.00,6151650000,-1.600218,174066.0
4,2020-05-19,54.344666,54.804668,53.738667,53.867332,53.867332,144547500,-0.878344,2948.59,2964.21,2922.35,2922.94,2922.94,4984330000,-0.869907,87362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,2022-11-29,184.990005,186.380005,178.750000,180.830002,180.830002,83357100,-2.248772,3964.19,3976.77,3937.65,3957.63,3957.63,3546040000,-0.165481,468782.0
223,2022-12-01,197.080002,198.919998,191.800003,194.699997,194.699997,80046200,-1.207634,4087.14,4100.51,4050.87,4076.57,4076.57,4527130000,-0.258616,313947.0
224,2023-03-31,197.529999,207.789993,197.199997,207.460007,207.460007,170222100,5.027089,4056.18,4110.75,4056.18,4109.31,4109.31,4525120000,1.309853,12219109.0
225,2023-05-17,168.410004,174.500000,167.190002,173.860001,173.860001,125473600,3.236148,4122.85,4164.67,4113.62,4158.77,4158.77,4039080000,0.871242,6721230.0


In [31]:
final_merged_df = final_merged_df[final_merged_df['Date'] >= '2020-05-13'].reset_index(drop=True)
final_merged_df

,Date,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,TSLA_Return,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Adj Close,GSPC_Volume,GSPC_Return,change_in_followers
0,2020-05-13,54.722000,55.066666,50.886665,52.730667,52.730667,285982500,-3.638999,2865.86,2874.14,2793.15,2820.00,2820.00,6151650000,-1.600218,174066.0
1,2020-05-19,54.344666,54.804668,53.738667,53.867332,53.867332,144547500,-0.878344,2948.59,2964.21,2922.35,2922.94,2922.94,4984330000,-0.869907,87362.0
2,2020-05-26,55.633331,55.639999,54.380669,54.591331,54.591331,121345500,-1.872978,3004.08,3021.72,2988.17,2991.77,2991.77,5848500000,-0.409776,68096.0
3,2020-05-28,54.234001,54.983334,53.445999,53.720669,53.720669,108834000,-0.946514,3046.61,3068.67,3023.40,3029.73,3029.73,5447020000,-0.554058,150412.0
4,2020-06-04,59.325333,59.716667,57.229332,57.625332,57.625332,133315500,-2.865556,3111.56,3128.91,3090.41,3112.35,3112.35,6451290000,0.025389,505668.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2022-11-29,184.990005,186.380005,178.750000,180.830002,180.830002,83357100,-2.248772,3964.19,3976.77,3937.65,3957.63,3957.63,3546040000,-0.165481,468782.0
220,2022-12-01,197.080002,198.919998,191.800003,194.699997,194.699997,80046200,-1.207634,4087.14,4100.51,4050.87,4076.57,4076.57,4527130000,-0.258616,313947.0
221,2023-03-31,197.529999,207.789993,197.199997,207.460007,207.460007,170222100,5.027089,4056.18,4110.75,4056.18,4109.31,4109.31,4525120000,1.309853,12219109.0
222,2023-05-17,168.410004,174.500000,167.190002,173.860001,173.860001,125473600,3.236148,4122.85,4164.67,4113.62,4158.77,4158.77,4039080000,0.871242,6721230.0


In [32]:
# create column for positive returns 
final_merged_df['TSLA_Positive_Return'] = (final_merged_df['TSLA_Return'] > 0).astype(int)

In [33]:
final_merged_df.to_csv('project_checkin_data.csv', index = False)

In [34]:
final_merged_df

,Date,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,TSLA_Return,GSPC_Open,GSPC_High,GSPC_Low,GSPC_Close,GSPC_Adj Close,GSPC_Volume,GSPC_Return,change_in_followers,TSLA_Positive_Return
0,2020-05-13,54.722000,55.066666,50.886665,52.730667,52.730667,285982500,-3.638999,2865.86,2874.14,2793.15,2820.00,2820.00,6151650000,-1.600218,174066.0,0
1,2020-05-19,54.344666,54.804668,53.738667,53.867332,53.867332,144547500,-0.878344,2948.59,2964.21,2922.35,2922.94,2922.94,4984330000,-0.869907,87362.0,0
2,2020-05-26,55.633331,55.639999,54.380669,54.591331,54.591331,121345500,-1.872978,3004.08,3021.72,2988.17,2991.77,2991.77,5848500000,-0.409776,68096.0,0
3,2020-05-28,54.234001,54.983334,53.445999,53.720669,53.720669,108834000,-0.946514,3046.61,3068.67,3023.40,3029.73,3029.73,5447020000,-0.554058,150412.0,0
4,2020-06-04,59.325333,59.716667,57.229332,57.625332,57.625332,133315500,-2.865556,3111.56,3128.91,3090.41,3112.35,3112.35,6451290000,0.025389,505668.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2022-11-29,184.990005,186.380005,178.750000,180.830002,180.830002,83357100,-2.248772,3964.19,3976.77,3937.65,3957.63,3957.63,3546040000,-0.165481,468782.0,0
220,2022-12-01,197.080002,198.919998,191.800003,194.699997,194.699997,80046200,-1.207634,4087.14,4100.51,4050.87,4076.57,4076.57,4527130000,-0.258616,313947.0,0
221,2023-03-31,197.529999,207.789993,197.199997,207.460007,207.460007,170222100,5.027089,4056.18,4110.75,4056.18,4109.31,4109.31,4525120000,1.309853,12219109.0,1
222,2023-05-17,168.410004,174.500000,167.190002,173.860001,173.860001,125473600,3.236148,4122.85,4164.67,4113.62,4158.77,4158.77,4039080000,0.871242,6721230.0,1
